In [39]:
import gymnasium as gym
from pogema import GridConfig

from stable_baselines3.common.evaluation import evaluate_policy

%load_ext autoreload
%autoreload 2
%matplotlib inline

grid_config = GridConfig(
    size=8,
    density=0.3,
    num_agents=1,
    max_episode_steps=30
)

env = gym.make("Pogema-v0",grid_config=grid_config)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/bernard/miniconda3/envs/rlenv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:29: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/home/bernard/miniconda3/envs/rlenv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:34: UserWarning: WARN: It seems a Box observation space is an image but the lower and upper bounds are not [0, 255]. Actual lower bound: -1.0, upper bound: 1.0. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(


### Optuna Integration

In [40]:
from typing import Any
from typing import Dict

import gymnasium
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
import torch
import torch.nn as nn


N_TRIALS = 200
N_STARTUP_TRIALS = 5
N_EVALUATIONS = 2
N_TIMESTEPS = int(1.2e5)
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_EPISODES = 3

DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
    "verbose": 1,
    "env": "Pogema-v0"
}

def sample_dqn_params(trial: optuna.Trial) -> Dict[str, Any]:
    """Sampler for DQN hyperparameters."""
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1, log=True)
    batch_size = 2 ** trial.suggest_int("batch_size", 3, 10)
    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 10.0, log=True)
    target_update_interval = trial.suggest_int("target_update_interval", 500, 20000, log=True)
    exploration_fraction = trial.suggest_float("exploration_fraction", 0.1, 0.5, log=True)
    exploration_final_eps = trial.suggest_float("exploration_final_eps", 0.01, 0.1, log=True)

    # Display true values.
    trial.set_user_attr("gamma", gamma)
    trial.set_user_attr("batch_size", batch_size)

    return {
        "learning_rate": learning_rate,        
        "batch_size": batch_size,
        "gamma": gamma,
        "max_grad_norm": max_grad_norm,
        "target_update_interval": target_update_interval,
        "exploration_fraction": exploration_fraction,
        "exploration_final_eps": exploration_final_eps
    }


class TrialEvalCallback(EvalCallback):
    """Callback used for evaluating and reporting a trial."""

    def __init__(
        self,
        eval_env: gymnasium.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):
        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            super()._on_step()
            self.eval_idx += 1
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need.
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True


def objective(trial: optuna.Trial) -> float:
    kwargs = DEFAULT_HYPERPARAMS.copy()
    # Sample hyperparameters.
    kwargs.update(sample_dqn_params(trial))
    # Create the RL model.
    model = DQN(**kwargs)
    # Create env used for evaluation.
    eval_env = Monitor(env)
    # Create the callback that will periodically evaluate and report the performance.
    eval_callback = TrialEvalCallback(
        eval_env, trial, n_eval_episodes=N_EVAL_EPISODES, eval_freq=EVAL_FREQ, deterministic=True
    )

    nan_encountered = False
    try:
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN.
        print(e)
        nan_encountered = True
    finally:
        # Free memory.
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed.
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward


if __name__ == "__main__":
    # Set pytorch num threads to 1 for faster training.
    torch.set_num_threads(1)

    sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
    # Do not prune before 1/3 of the max budget is used.
    pruner = MedianPruner(n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3)

    study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")
    try:
        study.optimize(objective, n_trials=N_TRIALS, timeout=600)
    except KeyboardInterrupt:
        pass

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    print("  User attrs:")
    for key, value in trial.user_attrs.items():
        print("    {}: {}".format(key, value))

[I 2023-11-04 19:57:38,826] A new study created in memory with name: no-name-da4dad63-1538-4fc3-954d-0155c63311da


Using cuda device
Creating environment from the given name 'Pogema-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 59.8     |
|    ep_rew_mean      | 0.25     |
|    exploration_rate | 0.995    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 13863    |
|    time_elapsed     | 0        |
|    total_timesteps  | 239      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 40.9     |
|    ep_rew_mean      | 0.5      |
|    exploration_rate | 0.994    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 12953    |
|    time_elapsed     | 0        |
|    total_timesteps  | 327      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_

/home/bernard/miniconda3/envs/rlenv/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:231: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 2.92GB > 2.22GB
  warnings.warn(


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 45.4     |
|    ep_rew_mean      | 0.458    |
|    exploration_rate | 0.957    |
| time/               |          |
|    episodes         | 48       |
|    fps              | 13847    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2178     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 44.5     |
|    ep_rew_mean      | 0.481    |
|    exploration_rate | 0.954    |
| time/               |          |
|    episodes         | 52       |
|    fps              | 13740    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2316     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 45.9     |
|    ep_rew_mean      | 0.446    |
|    exploration_rate | 0.949    |
| time/               |          |
|    episodes       

[I 2023-11-04 19:58:46,487] Trial 0 finished with value: 0.0 and parameters: {'learning_rate': 0.07106824614220245, 'batch_size': 6, 'gamma': 0.06583044475636637, 'max_grad_norm': 1.061792276936406, 'target_update_interval': 2113, 'exploration_fraction': 0.41448530539027345, 'exploration_final_eps': 0.015034610792265648}. Best is trial 0 with value: 0.0.


...

Using cuda device
Creating environment from the given name 'Pogema-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 64       |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.988    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 8739     |
|    time_elapsed     | 0        |
|    total_timesteps  | 256      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 57.4     |
|    ep_rew_mean      | 0.125    |
|    exploration_rate | 0.979    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 10219    |
|    time_elapsed     | 0        |
|    total_timesteps  | 459      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_

/home/bernard/miniconda3/envs/rlenv/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:231: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 2.92GB > 0.53GB
  warnings.warn(


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 51.5     |
|    ep_rew_mean      | 0.3      |
|    exploration_rate | 0.905    |
| time/               |          |
|    episodes         | 40       |
|    fps              | 10930    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2062     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 50.1     |
|    ep_rew_mean      | 0.318    |
|    exploration_rate | 0.898    |
| time/               |          |
|    episodes         | 44       |
|    fps              | 10457    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2204     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 49       |
|    ep_rew_mean      | 0.354    |
|    exploration_rate | 0.891    |
| time/               |          |
|    episodes       

[I 2023-11-04 20:09:34,883] Trial 7 finished with value: 0.0 and parameters: {'learning_rate': 0.0003461190480944761, 'batch_size': 10, 'gamma': 0.00010999235374153814, 'max_grad_norm': 3.638394295721496, 'target_update_interval': 8377, 'exploration_fraction': 0.16234737108489664, 'exploration_final_eps': 0.0983211872282596}. Best is trial 2 with value: 0.3333333333333333.


Number of finished trials:  8
Best trial:
  Value:  0.3333333333333333
  Params: 
    learning_rate: 1.8407920478653214e-05
    batch_size: 9
    gamma: 0.0007565366907137496
    max_grad_norm: 9.482308659624474
    target_update_interval: 1888
    exploration_fraction: 0.10891274494616725
    exploration_final_eps: 0.022033917540945417
  User attrs:
    gamma: 0.9992434633092863
    batch_size: 512


### Save tuned hyperparameters

In [41]:
SAVE_PARAMS_PATH = 'saved/tuned_params.yml'
SAVE_METRICS_PATH = 'saved/evaluation_metrics.yml'
MODEL_NAME = 'DQN'

In [42]:
from lib.utils import *

save_model_params(trial, MODEL_NAME, SAVE_PARAMS_PATH, DEFAULT_HYPERPARAMS)

### Evaluation

In [43]:
model_params = get_model_log(MODEL_NAME, SAVE_PARAMS_PATH)
model = load_model_params(DQN, model_params)

metrics = evaluate_metrics(model, env, num_episodes=30, num_trials=100, verbose=False)
save_metrics(metrics, MODEL_NAME, SAVE_METRICS_PATH)
print(f"Agent Success Rate: {metrics['success_rate'] * 100:.2f}%")
print(f"Steps to termination : {metrics['step_array']}")
print(f"Average steps to termination : {metrics['ave_steps']}")

Using cuda device
Creating environment from the given name 'Pogema-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/bernard/miniconda3/envs/rlenv/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:231: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 2.92GB > 0.29GB
  warnings.warn(


Agent Success Rate: 3.00%
Steps to termination : [2, 3, 1]
Average steps to termination : 2.0
